In [ ]:
import os, sys
import gc
import scipy as sp
from scipy.stats import norm
from scipy import linalg
import h5py
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler
from datetime import datetime
from scipy.stats import t
import matplotlib.pyplot as plt
from pandas_plink import read_plink1_bin
from statsmodels.stats.multitest import local_fdr
import random
import copy
import csv
import warnings
warnings.filterwarnings("ignore")

In [ ]:
experiments = ['34.1.1']

pops_list = ['EurOnly','EurEas','EurEasAmr','EurEasAmrAfr']

In [ ]:
for pops_trio in pops_list:
    
    print(pops_trio)
    
    for exp_num in experiments:
        
        #Read Auxiliary Population Summary Statistics
        ss_df_aux1 = pd.read_csv('/data/users/cs21d404/New2_Exp34/aux_one'+str(exp_num)+\
                                 '.sumstat', delimiter=' ', header=None,names=['chr','snp',\
                                'pos','effAllele','refAllele','beta','se','t','pvalue','n'])

        ss_df_aux2 = pd.read_csv('/data/users/cs21d404/New2_Exp34/aux_two'+str(exp_num)+\
                                 '.sumstat', delimiter=' ', header=None, names=['chr','snp',\
                                'pos','effAllele','refAllele','beta','se','t','pvalue','n'])

        ss_df_aux3 = pd.read_csv('/data/users/cs21d404/New2_Exp34/aux_three'+str(exp_num)+\
                                 '.sumstat', delimiter=' ', header=None, names=['chr','snp',\
                                'pos','effAllele','refAllele','beta','se','t','pvalue','n'])

        ss_df_aux4 = pd.read_csv('/data/users/cs21d404/New2_Exp34/aux_four'+str(exp_num)+\
                                 '.sumstat', delimiter=' ', header=None, names=['chr','snp',\
                                'pos','effAllele','refAllele','beta','se','t','pvalue','n'])

        #Read Target Population Summary Statistics
        ss_df_tar = pd.read_csv('/data/users/cs21d404/New2_Exp34/target'+str(exp_num)+\
                                '.sumstat', delimiter=' ', header=None, names=['chr','snp',\
                                'pos','effAllele','refAllele', 'beta','se','t','pvalue','n'])

        #Read Target Population LD Genotype Data (Ref./LD File)
        G_sas = read_plink1_bin("/data/users/cs21d404/g1000_sas/sas_chr22_geno.bed",\
                                "/data/users/cs21d404/g1000_sas/sas_chr22_geno.bim",\
                                "/data/users/cs21d404/g1000_sas/sas_chr22_geno.fam",\
                                verbose=False)
        X_sas = G_sas.values
        X_sas = (X_sas - np.mean(X_sas, axis=0))/np.std(X_sas, axis=0)
        sas_ld_full = X_sas
        all_snps_sas_ld_full = (G_sas.sel(sample='HG01583').snp).values

        common_snps = list(set(ss_df_aux1.iloc[:,1]) & set(ss_df_aux2.iloc[:,1]) &\
                           set(ss_df_aux3.iloc[:,1]) & set(ss_df_aux4.iloc[:,1]) &\
                           set(ss_df_tar.iloc[:,1]) & set(all_snps_sas_ld_full))

        #Process Data
        LDblocks_df = {
            'chr ':['chr22 '],
            'start': [0],
            'stop':[51243298]
        }
        LDblocks_df = pd.DataFrame(LDblocks_df)
        LDblocks_df_grouped = LDblocks_df.groupby('chr ')

        chrs=[22]

        #Preprocess Target and Auxiliary Summ. Stats. File
        ss_df_tar = ss_df_tar[ss_df_tar["snp"].isin(common_snps)]
        ss_df_aux1 = ss_df_aux1[ss_df_aux1["snp"].isin(common_snps)]
        ss_df_aux2 = ss_df_aux2[ss_df_aux2["snp"].isin(common_snps)]
        ss_df_aux3 = ss_df_aux3[ss_df_aux3["snp"].isin(common_snps)]
        ss_df_aux4 = ss_df_aux4[ss_df_aux4["snp"].isin(common_snps)]
        ss_df_tar_grouped = ss_df_tar.groupby('chr')
        N_tar = int(ss_df_tar.iloc[0,9])
        N_aux1 = int(ss_df_aux1.iloc[0,9])
        N_aux2 = int(ss_df_aux2.iloc[0,9])
        N_aux3 = int(ss_df_aux3.iloc[0,9])
        N_aux4 = int(ss_df_aux4.iloc[0,9])

        aux1_snps = []
        aux1_pvals = []
        aux1_betas = []
        aux1_chroms = []

        aux2_snps = []
        aux2_pvals = []
        aux2_betas = []
        aux2_chroms = []

        aux3_snps = []
        aux3_pvals = []
        aux3_betas = []
        aux3_chroms = []

        aux4_snps = []
        aux4_pvals = []
        aux4_betas = []
        aux4_chroms = []

        tar_snps = []
        tar_a0 = []
        tar_a1 = []
        tar_pvals = []
        tar_betas = []
        tar_chroms = []
        tar_pos = []
        tar_tvals = []
        tar_corr = []

        for i in range(len(chrs)):
            temp = ss_df_tar_grouped.get_group(chrs[i])
            temp0 = LDblocks_df_grouped.get_group(str('chr')+str(chrs[i])+str(' '))
            temp0.iloc[0,1] = 1
            temp0.iloc[temp0.shape[0]-1,2] = float("inf")
            for j in range(temp0.shape[0]):
                temp1 = temp[temp["pos"]>=temp0.iloc[j,1]]
                temp2 = temp1[temp1["pos"]<temp0.iloc[j,2]]
                tar_snps.append(list(temp2.iloc[:,1]))
                tar_a0.append(list(temp2.iloc[:,3]))
                tar_a1.append(list(temp2.iloc[:,4]))
                tar_pvals.append(np.asarray(list(temp2.iloc[:,8]), dtype='float'))
                tar_betas.append(np.asarray(list(temp2.iloc[:,5]), dtype='float'))
                tar_chroms.append(list(temp2.iloc[:,0]))
                tar_pos.append(np.asarray(list(temp2.iloc[:,2]), dtype='float'))
                tar_tvals.append(np.asarray(list(temp2.iloc[:,7]), dtype='float'))
                tar_corr.append(np.asarray(list(temp2.iloc[:,5]), dtype='float')*N_tar)
                temp_snps = list(temp2.iloc[:,1])

                temp_aux1 = ss_df_aux1[ss_df_aux1["snp"].isin(temp_snps)]
                aux1_snps.append(list(temp_aux1.iloc[:,1]))
                aux1_pvals.append(np.asarray(list(temp_aux1.iloc[:,8]), dtype='float'))
                aux1_betas.append(np.asarray(list(temp_aux1.iloc[:,5]), dtype='float'))
                aux1_chroms.append(list(temp_aux1.iloc[:,0]))

                temp_aux2 = ss_df_aux2[ss_df_aux2["snp"].isin(temp_snps)]
                aux2_snps.append(list(temp_aux2.iloc[:,1]))
                aux2_pvals.append(np.asarray(list(temp_aux2.iloc[:,8]), dtype='float'))
                aux2_betas.append(np.asarray(list(temp_aux2.iloc[:,5]), dtype='float'))
                aux2_chroms.append(list(temp_aux2.iloc[:,0]))

                temp_aux3 = ss_df_aux3[ss_df_aux3["snp"].isin(temp_snps)]
                aux3_snps.append(list(temp_aux3.iloc[:,1]))
                aux3_pvals.append(np.asarray(list(temp_aux3.iloc[:,8]), dtype='float'))
                aux3_betas.append(np.asarray(list(temp_aux3.iloc[:,5]), dtype='float'))
                aux3_chroms.append(list(temp_aux3.iloc[:,0]))

                temp_aux4 = ss_df_aux4[ss_df_aux4["snp"].isin(temp_snps)]
                aux4_snps.append(list(temp_aux4.iloc[:,1]))
                aux4_pvals.append(np.asarray(list(temp_aux4.iloc[:,8]), dtype='float'))
                aux4_betas.append(np.asarray(list(temp_aux4.iloc[:,5]), dtype='float'))
                aux4_chroms.append(list(temp_aux4.iloc[:,0]))

        #Preprocess Target Ref. file
        X_tar = []
        sd_tar_ref = []
        for i in range(len(tar_snps)):
            temp_set = set(tar_snps[i])
            temp_indices_tar = [i for i, e in enumerate(all_snps_sas_ld_full) if e in temp_set]
            all_snps_sas_ld_full = [i for i in all_snps_sas_ld_full if i in common_snps]
            tar_ld = sas_ld_full[:, temp_indices_tar]
            tar_ld = (tar_ld - np.mean(tar_ld, axis=0))/np.std(tar_ld, axis=0)
            X_tar.append(tar_ld)
            t3 = np.std(tar_ld, axis = 0)
            sd_tar_ref.append(t3)

        aux_betas = []
        for i in range(len(aux1_betas)):
            #Choose according to pops in aux pops
            if pops_trio == 'EurOnly':
                temp = (aux1_betas[i]*1.0) + (aux2_betas[i]*0.0) + (aux3_betas[i]*0.0) + (aux4_betas[i]*0.0)
            elif pops_trio == 'EasOnly':
                temp = (aux1_betas[i]*0.0) + (aux2_betas[i]*1.0) + (aux3_betas[i]*0.0) + (aux4_betas[i]*0.0)
            elif pops_trio == 'AmrOnly':
                temp = (aux1_betas[i]*0.0) + (aux2_betas[i]*0.0) + (aux3_betas[i]*1.0) + (aux4_betas[i]*0.0)
            elif pops_trio == 'AfrOnly':
                temp = (aux1_betas[i]*0.0) + (aux2_betas[i]*0.0) + (aux3_betas[i]*0.0) + (aux4_betas[i]*1.0)

            elif pops_trio == 'EurAmr':
                per_pop_weight = [0.5,0.5]
                temp = (aux1_betas[i]*per_pop_weight[0]) + (aux2_betas[i]*0.0) +\
                (aux3_betas[i]*per_pop_weight[1]) + (aux4_betas[i]*0.0)
            elif pops_trio == 'EurEas':
                per_pop_weight = [0.5,0.5]
                temp = (aux1_betas[i]*per_pop_weight[0]) + (aux2_betas[i]*per_pop_weight[1]) +\
                (aux3_betas[i]*0.0) + (aux4_betas[i]*0.0)
            elif pops_trio == 'EurAfr':
                per_pop_weight = [0.5,0.5]
                temp = (aux1_betas[i]*per_pop_weight[0]) + (aux2_betas[i]*0.0) +\
                (aux3_betas[i]*0.0) + (aux4_betas[i]*per_pop_weight[1])
            elif pops_trio == 'EasAmr':
                per_pop_weight = [0.5,0.5]
                temp = (aux1_betas[i]*0.0) + (aux2_betas[i]*per_pop_weight[0]) +\
                (aux3_betas[i]*per_pop_weight[1]) + (aux4_betas[i]*0.0)
            elif pops_trio == 'EasAfr':
                per_pop_weight = [0.5,0.5]
                temp = (aux1_betas[i]*0.0) + (aux2_betas[i]*per_pop_weight[0]) +\
                (aux3_betas[i]*0.0) + (aux4_betas[i]*per_pop_weight[1])
            elif pops_trio == 'AmrAfr':
                per_pop_weight = [0.5,0.5]
                temp = (aux1_betas[i]*0.0) + (aux2_betas[i]*0.0) +\
                (aux3_betas[i]*per_pop_weight[0]) + (aux4_betas[i]*per_pop_weight[1])

            elif pops_trio == 'EurEasAmr':
                per_pop_weight = [0.33,0.33,0.33]
                temp = (aux1_betas[i]*per_pop_weight[0]) + (aux2_betas[i]*per_pop_weight[1]) +\
                (aux3_betas[i]*per_pop_weight[2]) + (aux4_betas[i]*0.0)
            elif pops_trio == 'EurEasAfr':
                per_pop_weight = [0.33,0.33,0.33]
                temp = (aux1_betas[i]*per_pop_weight[0]) + (aux2_betas[i]*per_pop_weight[1]) +\
                (aux3_betas[i]*0.0) + (aux4_betas[i]*per_pop_weight[2])
            elif pops_trio == 'EurAmrAfr':
                per_pop_weight = [0.33,0.33,0.33]
                temp = (aux1_betas[i]*per_pop_weight[0]) + (aux2_betas[i]*0.0) +\
                (aux3_betas[i]*per_pop_weight[1]) + (aux4_betas[i]*per_pop_weight[2])
            elif pops_trio == 'EasAmrAfr':
                per_pop_weight = [0.33,0.33,0.33]
                temp = (aux1_betas[i]*0.0) + (aux2_betas[i]*per_pop_weight[0]) +\
                (aux3_betas[i]*per_pop_weight[1]) + (aux4_betas[i]*per_pop_weight[2])

            elif pops_trio == 'EurEasAmrAfr':
                per_pop_weight = [0.25,0.25,0.25,0.25]
                temp = (aux1_betas[i]*per_pop_weight[0]) + (aux2_betas[i]*per_pop_weight[1]) + \
                (aux3_betas[i]*per_pop_weight[2]) + (aux4_betas[i]*per_pop_weight[3])
            else:
                print("Invalid Aux Pops!")

            aux_betas.append(temp)

        P = len(common_snps)
        num_chunks = len(tar_snps)

        #Initialize Beta_hats
        Beta_hat_init = []
        for i in range(num_chunks):
            tt = np.random.normal(0, 0.00000000001, len(tar_snps[i]))
            #tt = tar_betas[i]
            Beta_hat_init.append(tt)

#         L1_penalty = 6
#         L2_penalty = 0.00005
        
        L1_penalty = 1000
        L2_penalty = 0.00000005

        XtX = np.matmul(X_tar[0].T, X_tar[0])
        XtX = abs(1-L2_penalty)*XtX
        r = tar_corr[0]
        
        mu = 0.1

        def func(Bs, XtX, r, L1_penalty, L2_penalty):
            term1 = np.matmul(np.matmul(Bs.T, XtX), Bs)
            term2 = 2*np.matmul(Bs.T, tar_corr[0])
            term3 = L2_penalty*np.matmul(Bs.T,Bs)
            temp = Bs - np.concatenate(aux_betas)
            nesterov = mu*np.log((0.5*np.exp(-temp/mu))+(0.5*np.exp(temp/mu)))
            term4 = L1_penalty*sum(nesterov)
            return term1-term2+term3+term4

        def jac_func(Bs, XtX, r, L1_penalty, L2_penalty):
            term1 = 2*np.matmul(XtX,Bs)
            term2 = 2*r
            term3 = L2_penalty*2*Bs
            temp = Bs - np.concatenate(aux_betas)
            nesterov = np.divide((-np.exp(-temp/mu) + np.exp(temp/mu)),(np.exp(-temp/mu) + np.exp(temp/mu)))
            term4 = L1_penalty*nesterov
            return term1-term2+term3+term4
        
        bounds = [(-2,2)] * Beta_hat_init[0].shape[0]
        
        ff = [func(Beta_hat_init[0], XtX, r, L1_penalty, L2_penalty)]
        
        def callbackF(x):
            global ff
            ff.append(func(x, XtX, r, L1_penalty, L2_penalty))
        
        print(datetime.now())
        ans = sp.optimize.minimize(func, jac=jac_func, x0=np.concatenate(Beta_hat_init),\
                                   callback=callbackF,args=(XtX, r, L1_penalty, L2_penalty),method='L-BFGS-B')
        print(datetime.now())
        
        final_snps = np.concatenate(tar_snps)
        final_betas = ans.x
        final_corr = np.concatenate(tar_corr)
        final_chr = np.concatenate(tar_chroms)
        final_pos = np.concatenate(tar_pos)
        final_a0 = np.concatenate(tar_a0)
        final_a1 = np.concatenate(tar_a1)
        final_results_df = pd.DataFrame({'SNP':final_snps, 'BETA':final_betas, 'CORR':final_corr,\
                                         'CHR':final_chr, 'POS':final_pos, 'A0':final_a0, 'A1':final_a1})

        final_results_df.to_csv('/data/users/cs21d404/New2_Exp34/Results/mppV2_pred_tar_betas'+\
                                '_'+str(pops_trio)+str(exp_num)+'.txt', sep='\t', index=False)

        print(exp_num+" ... Done!")